In [1]:
pip install dash plotly librosa



[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import librosa
import numpy as np

# Load audio and MFCCs
y, sr = librosa.load('assets/Munbe Vaa Violin - 1_1.wav', sr=8000, mono=True)
mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Graph(
        id='mfcc-plot',
        figure={
            'data': [
                go.Heatmap(
                    z=mfccs,
                    colorscale='Viridis'
                )
            ],
            'layout': go.Layout(
                title='MFCCs of Master Audio',
                xaxis=dict(title='Time Frame'),
                yaxis=dict(title='MFCC Coefficients'),
                hovermode='closest'
            )
        }
    ),
    html.Audio(id='audio-playback', src='/assets/Munbe Vaa Violin - 1_1.wav', controls=True)
])

if __name__ == '__main__':
    app.run_server(debug=True)


In [19]:
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import librosa
import numpy as np

# Load audio and MFCCs
y, sr = librosa.load('assets/Munbe Vaa Violin - 1_1.wav', sr=8000, mono=True)
mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=1)

app = dash.Dash(__name__)

# Generating multiple scatter plots, one for each MFCC coefficient
traces = []
for i, mfcc in enumerate(mfccs):
    traces.append(
        go.Scatter(
            x=list(range(len(mfcc))),
            y=mfcc,
            mode='lines+markers',
            name=f'MFCC {i}'
        )
    )

app.layout = html.Div([
    dcc.Graph(
        id='mfcc-plot',
        figure={
            'data': traces,
            'layout': go.Layout(
                title='MFCCs of Master Audio',
                xaxis=dict(title='Time Frame'),
                yaxis=dict(title='MFCC Value'),
                hovermode='closest'
            )
        }
    ),
    html.Audio(id='audio-playback', src='/assets/Munbe Vaa Violin - 1_1.wav', controls=True)

])

if __name__ == '__main__':
    app.run_server(debug=True)


---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
File /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/dash/dash.py:1255, in Dash.dispatch(self=<dash.dash.Dash object>)
   1254 try:
-> 1255     cb = self.callback_map[output]
        output = 'audio-time.children'
        self.callback_map = {}
        self = <dash.dash.Dash object at 0x2a433be80>
   1256     func = cb["callback"]

KeyError: 'audio-time.children'

The above exception was the direct cause of the following exception:

KeyError                                  Traceback (most recent call last)
File /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/flask/app.py:1823, in Flask.full_dispatch_request(self=<Flask '__main__'>)
   1821     rv = self.preprocess_request()
   1822     if rv is None:
-> 1823         rv = self.dispatch_request()
        rv = None
        se

In [18]:
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import librosa
import numpy as np

# Load audio and MFCCs
y, sr = librosa.load('assets/Munbe Vaa Violin - 1_1.wav', sr=8000, mono=True)
mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=2)


app = dash.Dash(__name__)

# App layout
app.layout = html.Div([
    dcc.Graph(id='mfcc-plot'),
    html.Audio(id='audio-playback', src='/assets/Munbe Vaa Violin - 1_1.wav', controls=True),
    html.Div(id='audio-time', style={'display': 'none'}),
    dcc.Interval(id='interval-component', interval=1*1000, n_intervals=0)
])

@app.callback(
    Output('audio-time', 'children'),
    Input('interval-component', 'n_intervals')
)
def update_time(n):
    return [
        dcc.Store(id='store-audio-time', data=0),
        """
        <script>
            let audioElement = document.getElementById("audio-playback");
            document.getElementById("store-audio-time").data = audioElement.currentTime;
        </script>
        """
    ]

@app.callback(
    Output('mfcc-plot', 'figure'),
    Input('store-audio-time', 'data')
)
def update_graph(current_time):
    # Your logic to update the graph based on current time.
    traces = [
        go.Scatter(
            x=list(range(len(mfcc))),
            y=mfcc,
            mode='lines+markers',
            name=f'MFCC {i}'
        ) for i, mfcc in enumerate(mfccs)
    ]
    layout = go.Layout(
        title='MFCCs of Master Audio',
        xaxis=dict(title='Time Frame'),
        yaxis=dict(title='MFCC Value'),
        hovermode='closest',
        shapes=[
            {
                'type': 'line',
                'x0': current_time * sr / 512,  # Convert time to frame number (assuming hop length of 512)
                'x1': current_time * sr / 512,
                'y0': -600,
                'y1': 0,
                'line': {'color': 'red', 'width': 2}
            }
        ]
    )
    return {'data': traces, 'layout': layout}

if __name__ == "__main__":
    app.run_server(debug=True)
